## GAAIMS: Predicting Multiple Sclerosis from Dynamics of Gait Variability Using an Instrumented Treadmill - A Machine Learning-Based Approach
## Regression coefficients on controls for Trial W only - Computing the gait features for 30 controls on Trial W

### Package imports

In [97]:
import numpy as np
import pandas as pd
import math
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [98]:
path = 'C:\\Users\\Rachneet Kaur\\Dropbox\\GAIT\\control_data\\Treadmill_Data_Controls\\Treadmill_Data_Controls\\'

In [99]:
# Get all the file names in the dictionary
control_ids = [102, 104, 105, 107, 109, 111, 112, 113, 114, 115, 116, 117, 118, 120,
              122, 123, 124, 125, 126, 128, 129, 132, 133, 134, 136, 137, 138, 139, 140, 141] #30 controld - trial W data 
print (len(control_ids))
raw_controls = [path + 'FitNIRS_'+str(i)+'_WA_NA_RAWDATA.csv' for i in control_ids] #Rawdata 
gait_controls = [path + 'FitNIRS_'+str(i)+'_WA_NA_GAITCYCLES.csv' for i in control_ids] #GaitCycles

30


In [100]:
# for every GaitCycle file, a sequence of walk will always start with a heel strike on the right foot.
# Thus the order of the Gait event points would be HSR, TOL, MidSSR, HSL, TOR and MidSSL.
gait_type = np.array(['HSR', 'TOL', 'MidSSR', 'HSL', 'TOR', 'MidSSL'])

#Delta_time
delta_time = 0.002 #Since the data is collected is 500Hz frequency 

### Utility functions

In [101]:
#functions to drop missing values and invalid data 
def drop_unnamed(dataframe):
    return(dataframe.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1))

#Eliminate missing values
def drop_na(dataframe):
    return(pd.DataFrame.dropna(dataframe))

#Eliminate invalid data 
def get_valid(dataframe):
    return(dataframe.loc[dataframe.Valid == True, :])

# Valid strides in the gait_cycles.csv file 
def get_cycle(dataframe):
    stride_start = min(dataframe.loc[dataframe.EventType == 'HSR'].index)
    stride_end = max(dataframe.loc[dataframe.EventType == 'MidSSL'].index)   
    return dataframe.loc[stride_start:stride_end]

# Restore the indexing for the cropped dataframe 
def change_index(dataframe):
    dataframe.index = range(len(dataframe))
    return dataframe

# get all the valid index in order: HSR-TOL-MidSSR-HSL-TOR-MidSSL
def set_complete(data_frame):
    # input is the Dataframe includes ONLY valid points 
    # get all the index of HSR since it starts with heal strike left
    # if the length of last gait cycle contain HSR does not equals to 6, then ignore it
    
    HSR = data_frame.loc[data_frame.EventType == 'HSR'].index
    last_idx = HSR[-1]
    last_all_idx = data_frame.index[-1]
    # if the last gait cycles contains HSR is not a valid gait cycle, then we should consider the last second HSR instead.
    if((last_all_idx-last_idx) < 5):
        HSR = HSR[0:-1] 
    else:
        HSR = HSR
    
    # get all the valid index in order: HSR-TOL-MidSSR-HSL-TOR-MidSSL
    valid = []
    for idx_HSR in HSR:
        if (((idx_HSR + 1) in data_frame.index) & ((idx_HSR + 2) in data_frame.index) &
            ((idx_HSR + 3) in data_frame.index) & ((idx_HSR + 4) in data_frame.index) & 
            ((idx_HSR + 5) in data_frame.index)):
            # the valid index exist in the dataframe.
            if((data_frame.loc[idx_HSR + 1].EventType == 'TOL') & (data_frame.loc[idx_HSR + 2].EventType == 'MidSSR') & 
               (data_frame.loc[idx_HSR + 3].EventType == 'HSL') & (data_frame.loc[idx_HSR + 4].EventType == 'TOR') & 
               (data_frame.loc[idx_HSR + 5].EventType == 'MidSSL')):
                valid.extend(range(idx_HSR, idx_HSR+6))
    #returns the list of valid indices which form complete strides 
    return valid

### Data Preprocessing

In [102]:
#Preprocessing the files to delete missing and invalid data 
#For each person (control and MS) in Trial 1
def cleaning(pid):
    gait = pd.read_csv(gait_controls[pid], header = 1)
    raw = pd.read_csv(raw_controls[pid], header = 1) 
    print ('Time recorded for PID ', pid, ' is ', raw.dropna().Time.iloc[-1])
    gait = drop_na(gait)
    gait  = get_valid(gait)

    #Reducing to complete strides data 
    gait = get_cycle(gait)
    indices_complete = set_complete(gait)
    gait = gait.loc[indices_complete]

    #Resetting the index 
    gait = change_index(gait)
    return indices_complete, gait, raw

### Our gait cycle would be HSR, TOL, MidSSR, HSL, TOR and MidSSL

### Supporting times

In [103]:
#Supporting Times
#Double support: HSR-TOL
#Single support (Right): TOL-MidSSR-HSL
#Double Support: HSL-TOR
#Signle support (Left): TOR - MidSSL-HSR (of the next stride)
# Note, for counting supporting time of a foot for current stride, we need the HSR for the next stride
def get_cycle_double_single(Dataframe):
    stride_start = min(Dataframe.loc[Dataframe.EventType == 'HSR'].index)
    stride_end = max(Dataframe.loc[Dataframe.EventType == 'HSR'].index)   
    return Dataframe.loc[stride_start:stride_end]

# delete the 'mid' points for calculating supporting time for convenience
def delete_mid(Dataframe):
    midl = Dataframe.loc[Dataframe.EventType == 'MidSSL'].index
    midr = Dataframe.loc[Dataframe.EventType == 'MidSSR'].index
    new_index = pd.Int64Index(np.arange(len(Dataframe))).difference(list(midl) + list(midr))
    return(Dataframe.loc[pd.Int64Index(list(new_index))])

#This function computes the 4 features for supporting times ,namely, double support (on right and left heels) and single support 
#(on left and right foot) 
def support(gait):
    ####################
    #insert the support#
    ####################
    double_single = get_cycle_double_single(gait) 
    #Reducing the dataframe from first HSR to last HSR for calculating supporting times
    
    # change the index again for counting strides
    double_single = change_index(double_single)
    
    # MidSSR and MidSSL is useless for calculating the support time
    double_single = delete_mid(double_single)
    # get the time
    time = list(double_single['Time'])
    time_d_s = list(np.array(time[1:]) - np.array(time[0:-1])) 
    #Since now the events are HSR-TOL-HSL-TOR, we can simply take time[1:]-time[:-1]
    
    Double_LeftWhole_RightHeal = time_d_s[0::4] # support by whole left foot and right heal #HSR-TOL
    Single_Right = time_d_s[1::4] # support bi single right feet #TOL-HSL
    Double_RightWhole_LeftHeal = time_d_s[2::4] #HSL-TOR
    Single_left = time_d_s[3::4] #TOR-HSR (of the next stride)
    return Double_LeftWhole_RightHeal, Single_Right, Double_RightWhole_LeftHeal, Single_left

### Treadmill self-controlled speed and Ground reaction forces at gait events 

In [104]:
#Function returning the treadmill speed or ground reaction forces at 6 gait events 
def tspeeds_forces(gait, raw, stride_count, feature = 'Speed'):
    #Exact times of HSR
    HSR_times = gait['Time'][gait.EventType == 'HSR']
    #Exact times of TOR 
    TOR_times = gait['Time'][gait.EventType == 'TOR']
    #Exact times of HSL
    HSL_times = gait['Time'][gait.EventType == 'HSL']
    #Exact times of TOL 
    TOL_times = gait['Time'][gait.EventType == 'TOL']
    #Exact times of MidSSR 
    MidSSR_times = gait['Time'][gait.EventType == 'MidSSR']
    #Exact times of MidSSL
    MidSSL_times = gait['Time'][gait.EventType == 'MidSSL']

    #For six events of interest, calculate the closest times from RAWDATA.csv file 
    #and keep the treadmill speed (tspeed) at that point if feature == 'Speed' or 
    #ground reaction force if feature == 'TreadMill_FZ'
    HSR_raw = [raw[feature][raw['Time']>HSR_times.iloc[i]].iloc[0] for i in range(stride_count)]
    TOR_raw = [raw[feature][raw['Time']>TOR_times.iloc[i]].iloc[0] for i in range(stride_count)]
    HSL_raw = [raw[feature][raw['Time']>HSL_times.iloc[i]].iloc[0] for i in range(stride_count)]
    TOL_raw = [raw[feature][raw['Time']>TOL_times.iloc[i]].iloc[0] for i in range(stride_count)]
    MidSSR_raw = [raw[feature][raw['Time']>MidSSR_times.iloc[i]].iloc[0] for i in range(stride_count)]
    MidSSL_raw = [raw[feature][raw['Time']>MidSSL_times.iloc[i]].iloc[0] for i in range(stride_count)]

    return HSR_raw, MidSSR_raw, TOR_raw, HSL_raw, TOL_raw, MidSSL_raw

### Stride, swing and stance times for each stride 

In [105]:
#Function computing the stride, swing and stance times for each stride 
def times(gait):
    #Exact times of HSR
    HSR_times = gait['Time'][gait.EventType == 'HSR']
    #Exact times of TOR 
    TOR_times = gait['Time'][gait.EventType == 'TOR']
    #Exact times of HSL
    HSL_times = gait['Time'][gait.EventType == 'HSL']
    #Exact times of TOL 
    TOL_times = gait['Time'][gait.EventType == 'TOL']
    #Exact times of MidSSR 
    MidSSR_times = gait['Time'][gait.EventType == 'MidSSR']
    #Exact times of MidSSL
    MidSSL_times = gait['Time'][gait.EventType == 'MidSSL']

    #Stride Time = Next HSR Time - Current HSR Time
    stride_times = HSR_times[1:].values - HSR_times[:-1].values

    #Swing time = Next HSR time - Current TOR time
    swing_times = HSR_times[1:].values - TOR_times[:-1].values

    #Stance time = Current TOR time - Current HSR time 
    stance_times = TOR_times.values - HSR_times.values

    return stride_times, swing_times, stance_times

### Stride length

In [106]:
#Function returning the difference between consequetive Y-coordinates 
def stride_len(y1, y2):
    return y2-y1

def length(gait, raw, indices_complete, stride_count):
    #Function returning the stride length 
    #Exact times of HSR
    HSR_times = gait['Time'][gait.EventType == 'HSR']

    #For HSR, calculate the closest times from RAWDATA.csv file 
    HSR_times_raw = [raw['Time'][raw['Time']>HSR_times.iloc[i]].iloc[0] for i in range(stride_count)]

    #Y for HSR
    HSR_Y = gait['Y'][gait.EventType == 'HSR']
    rely_progR = []

    for idx in range(0, stride_count): #Use for all strides for each person, each trial
        try:
            #For Right Foot
            #Relative y indices for HSR(i-1) to HSR(i) 
            rely_prog_idxR = (raw['Time']>=HSR_times_raw[idx]) & (raw['Time']<HSR_times_raw[idx+1]) #Progression vector 
            #Relative_y or Belt Speed = Speed*dt = Area under the speed curve *dt for HSR(i-1) to HSR(i)
            rely_progR.append(np.trapz(raw['Speed'][rely_prog_idxR])*0.002) 
        except:
            pass
    #Right Foot 
    #HSR_Y after adding the relative y correspoding to previous HSR
    rel_HSR_Y = HSR_Y[1:].values+np.array(rely_progR) 

    #Stride length HSR-NextHSR 
    length = np.array(list(map(stride_len, HSR_Y[:-1].values, rel_HSR_Y)))

    #Right Foot
    #Convert in-consequetive gait cycles' stride length to NaN 
    stride_idx = np.array(indices_complete[::6][1:]) - np.array(indices_complete[::6][:-1])
    #If this difference is not 6, that means the valid strides is not in consequent order, hence, we cannot compute lengths 
    length[np.where(stride_idx!=6)[0]] = np.nan
    return length

### Stride width

In [107]:
#Stride Width = abs((x2-x1)*(y1-y0) - (x1-x0)*(y2-y1)) / np.sqrt(np.square(x2-x1) + np.square(y2-y1))
#where (x0, y0) is the point from HS of opposite feet (i.e. coordinates of HSL)
#and (x1,y1), (x2,y2) are coordinates that make the line joining HS(i-1) and HS(i) of same feet
def stride_wid(x0, y0, x1, y1, x2, y2):
    return np.abs((x2-x1)*(y1-y0) - (x1-x0)*(y2-y1)) / np.sqrt((x2-x1)**2 + (y2-y1)**2)

In [108]:
#Function returning the stride width
def calc_width(gait, raw, indices_complete, stride_count):
    #Exact times of HSR
    HSR_times = gait['Time'][gait.EventType == 'HSR']
    #Exact times of HSL
    HSL_times = gait['Time'][gait.EventType == 'HSL']

    #For HSR, calculate the closest times from RAWDATA.csv file 
    HSR_times_raw = [raw['Time'][raw['Time']>HSR_times.iloc[i]].iloc[0] for i in range(stride_count)]
    #For HSL, calculate the closest times from RAWDATA.csv file 
    HSL_times_raw = [raw['Time'][raw['Time']>HSL_times.iloc[i]].iloc[0] for i in range(stride_count)]

    #Y for HSR
    HSR_Y = gait['Y'][gait.EventType == 'HSR']
    #Y for HSL
    HSL_Y = gait['Y'][gait.EventType == 'HSL']

    #X for HSR
    HSR_X = gait['X'][gait.EventType == 'HSR']
    #X for HSL
    HSL_X = gait['X'][gait.EventType == 'HSL']

    rely_progR = []
    rely_progL = []

    for idx in range(0, stride_count): #Use for all strides for each person, each trial
        try:
            #For Right Foot
            #Relative y indices for HSR(i-1) to HSR(i) 
            rely_prog_idxR = (raw['Time']>=HSR_times_raw[idx]) & (raw['Time']<HSR_times_raw[idx+1]) #Progression vector 
            #Relative_y or Belt Speed = Speed*dt = Area under the speed curve *dt for HSR(i-1) to HSR(i)
            rely_progR.append(np.trapz(raw['Speed'][rely_prog_idxR])*0.002) 

            #Relative y indices for HSR(i-1) to HSL(i-1) 
            rely_prog_idxL = (raw['Time']>=HSR_times_raw[idx]) & (raw['Time']<HSL_times_raw[idx]) #Progression vector 
            #Relative_y or Belt Speed = Speed*dt = Area under the speed curve *dt for HSR(i-1) to HSL(i-1)
            rely_progL.append(np.trapz(raw['Speed'][rely_prog_idxL])*0.002)         

        except:
            pass
    #Right Foot 
    #HSR_Y after adding the relative y correspoding to previous HSR
    rel_HSR_Y = HSR_Y[1:].values+np.array(rely_progR) 
    #Left Foot 
    #HSL_Y after adding the relative y correspoding to same stride's HSR
    rel_HSL_Y = HSL_Y[:-1].values+np.array(rely_progL) 

    #Stride width HSR-HSL-NextHSR 
    width = np.array(list(map(stride_wid, HSL_X[:-1].values, rel_HSL_Y, HSR_X[:-1].values, HSR_Y[:-1].values, 
                                     HSR_X[1:].values, rel_HSR_Y)))

    #Right Foot
    #Convert in-consequetive gait cycles' stride width to NaN 
    stride_idx = np.array(indices_complete[::6][1:]) - np.array(indices_complete[::6][:-1])
    #If this difference is not 6, that means the valid strides is not in consequent order, hence, we cannot compute lengths 
    width[np.where(stride_idx!=6)[0]] = np.nan
    return width

### Computing all the gait-based features

In [109]:
#Appending the 24 features 
def gait_features(pid):
    df = pd.DataFrame()
    indices_complete, gait, raw = cleaning(pid)
    stride_count = int(gait.shape[0]/6) #6 events in each stride
    
    #Inserting the supporting times 
    DS_R, SS_R, DS_L, SS_L = support(gait)
    #Total length must match the stride count 
    #Append NaN at the end for all the supporting times since for SS_L, we need the next stride available, 
    #hence SS_L does not exist for the last stride
    df['DS_R'], df['SS_R'], df['DS_L'], df['SS_L'] = np.append(DS_R, np.nan), np.append(SS_R, np.nan), np.append(DS_L, np.nan), np.append(SS_L, np.nan)
    
    #Inserting the treadmill speeds 
    df['tspeed_HSR'], df['tspeed_MidSSR'], df['tspeed_TOR'], df['tspeed_HSL'], df['tspeed_TOL'], df['tspeed_MidSSL'] = tspeeds_forces(gait, raw, stride_count, 'Speed')
    
    #Inserting the ground reaction forces 
    df['force_HSR'], df['force_MidSSR'], df['force_TOR'], df['force_HSL'], df['force_TOL'], df['force_MidSSL'] = tspeeds_forces(gait, raw, stride_count, 'TreadMill_FZ')
    
    #Inserting the stride, stance and swing times 
    stride_time, swing_time, stance_time= times(gait)
    #Append NaN at the end for all the stride and swing time since we need next stride for computation of these at current stride
    df['stride_time'], df['swing_time'] = np.append(stride_time, np.nan), np.append(swing_time, np.nan)
    df['stance_time'] = stance_time
    
    #Inserting the stride length 
    stride_length = length(gait, raw, indices_complete, stride_count)
    #For length, append NaN at the end
    df['stride_length'] = np.append(stride_length, np.nan)
    
    #Inserting the stride width
    stride_width = calc_width(gait, raw, indices_complete, stride_count)
    #For stride width, append NaN at the end 
    df['stride_width'] = np.append(stride_width, np.nan)
    
    #Inserting the stride speed
    df['stride_speed'] = df['stride_length']/df['stride_time']
    
    #Inserting the cadence (steps per minute i.e. 60*2/stride_time since 1 stride has 2 steps 
    #and stride time is in seconds so multiple by 60 to compute steps in a minute)
    df['cadence'] = 60*2/df['stride_time'] 
    
    #Inserting the walk ratio = sride_length/(strides per minute) where stride_per_min = cadence/2 (Unit: m/strides/min)
    df['walk_ratio'] = 2*df['stride_length']/df['cadence']
    return df

In [110]:
#Dataframe with Patient ID, Trial ID, (Right FPA, Left FPA for each stride)
final_df = pd.DataFrame()

for index in range(0, 30): #30 controls for trial W
    pid = control_ids[index]
    df = gait_features(index)

    temp_df = pd.DataFrame(data = np.array([[pid]*len(df)]).T)
    temp_df.columns = ['PID']
    temp_df = pd.concat([temp_df, df], axis = 1)
    final_df = final_df.append(temp_df, ignore_index= True)

Time recorded for PID  0  is  177.737999995797
Time recorded for PID  1  is  151.326000030829
Time recorded for PID  2  is  173.248000025191
Time recorded for PID  3  is  154.892000031556
Time recorded for PID  4  is  168.88599999700898
Time recorded for PID  5  is  151.83399999641
Time recorded for PID  6  is  151.979999996406
Time recorded for PID  7  is  169.98599999598
Time recorded for PID  8  is  155.390000017652
Time recorded for PID  9  is  153.972000031368
Time recorded for PID  10  is  180.95199999572102
Time recorded for PID  11  is  141.199999996661
Time recorded for PID  12  is  168.19199999602301
Time recorded for PID  13  is  152.349999996397
Time recorded for PID  14  is  171.123999995953
Time recorded for PID  15  is  200.94799999524798
Time recorded for PID  16  is  178.44999999578
Time recorded for PID  17  is  171.06999999595502
Time recorded for PID  18  is  216.238000001033
Time recorded for PID  19  is  197.54599999532903
Time recorded for PID  20  is  166.065999

In [111]:
final_df.head()

,PID,DS_R,SS_R,DS_L,SS_L,tspeed_HSR,tspeed_MidSSR,tspeed_TOR,tspeed_HSL,tspeed_TOL,...,force_TOL,force_MidSSL,stride_time,swing_time,stance_time,stride_length,stride_width,stride_speed,cadence,walk_ratio
0,102,0.214,0.496,0.236,0.504,0.853843,0.832884,0.794530,0.811716,0.849651,...,865.859765,805.434696,1.450,0.504,0.946,1.404388,0.075299,0.968544,82.758621,0.033939
1,102,0.270,0.334,0.190,0.296,0.865579,0.963664,1.046869,0.991748,0.926358,...,891.600148,829.797523,1.090,0.296,0.794,0.808027,0.150372,0.741309,110.091743,0.014679
2,102,0.180,0.438,0.214,0.418,1.046450,1.025072,1.014803,0.992796,1.024234,...,949.259385,805.615205,1.250,0.418,0.832,1.132094,0.079525,0.905675,96.000000,0.023585
3,102,0.248,0.370,0.262,0.356,1.019204,0.997617,0.976449,1.002437,1.033665,...,914.069152,825.344213,1.236,0.356,0.880,1.017947,0.158556,0.823582,97.087379,0.020970
4,102,0.218,0.430,0.222,0.436,0.949622,0.901628,0.883394,0.910011,0.935580,...,908.027960,825.620958,1.306,0.436,0.870,0.876773,0.173470,0.671342,91.883614,0.019084


### Butterfly diagram based feature extraction

In [112]:
"""
Reference: https://github.com/sukhbinder/intersection/blob/master/intersection.py
Sukhbinder
5 April 2017
"""

def _rect_inter_inner(x1,x2):
    n1=x1.shape[0]-1
    n2=x2.shape[0]-1
    X1=np.c_[x1[:-1],x1[1:]]
    X2=np.c_[x2[:-1],x2[1:]]
    S1=np.tile(X1.min(axis=1),(n2,1)).T
    S2=np.tile(X2.max(axis=1),(n1,1))
    S3=np.tile(X1.max(axis=1),(n2,1)).T
    S4=np.tile(X2.min(axis=1),(n1,1))
    return S1,S2,S3,S4

def _rectangle_intersection_(x1,y1,x2,y2):
    S1,S2,S3,S4=_rect_inter_inner(x1,x2)
    S5,S6,S7,S8=_rect_inter_inner(y1,y2)

    C1=np.less_equal(S1,S2)
    C2=np.greater_equal(S3,S4)
    C3=np.less_equal(S5,S6)
    C4=np.greater_equal(S7,S8)

    ii,jj=np.nonzero(C1 & C2 & C3 & C4)
    return ii,jj

def intersection(x1,y1,x2,y2):
    '''
    INTERSECTIONS Intersections of curves.
       Computes the (x,y) locations where two curves intersect.  The curves
       can be broken with NaNs or have vertical segments.
    usage:
    x,y=intersection(x1,y1,x2,y2)
    '''
    ii,jj=_rectangle_intersection_(x1,y1,x2,y2)
    n=len(ii)

    dxy1=np.diff(np.c_[x1,y1],axis=0)
    dxy2=np.diff(np.c_[x2,y2],axis=0)

    T=np.zeros((4,n))
    AA=np.zeros((4,4,n))
    AA[0:2,2,:]=-1
    AA[2:4,3,:]=-1
    AA[0::2,0,:]=dxy1[ii,:].T
    AA[1::2,1,:]=dxy2[jj,:].T

    BB=np.zeros((4,n))
    BB[0,:]=-x1[ii].ravel()
    BB[1,:]=-x2[jj].ravel()
    BB[2,:]=-y1[ii].ravel()
    BB[3,:]=-y2[jj].ravel()

    for i in range(n):
        try:
            T[:,i]=np.linalg.solve(AA[:,:,i],BB[:,i])
        except:
            T[:,i]=np.NaN


    in_range= (T[0,:] >=0) & (T[1,:] >=0) & (T[0,:] <=1) & (T[1,:] <=1)

    xy0=T[2:,in_range]
    xy0=xy0.T
    return xy0[:,0],xy0[:,1]

### Generating butterfly diagrams 

In [117]:
#For generating the butterfly diagrams for Controls, PwMS Trial W and Trial WT
def butterfly_plots(pid):
    indices_complete, gait, raw = cleaning(pid)

    #For the four events of interest for butterfly diagram, computing the times from GAITCYCLES.csv file
    # HSR-TOL-MidSSR-HSL-TOR-MidSSL
    #Line 1 is COPs from HSR-TOL, Line 2 is COPs from HSL-TOR
    #gait = gait[(gait['Time']>30) & (gait['Time']<60)] 
    #Taking only 30-60 seconds of the trial for cleaner results for mean trajectory 
    stride_count = int(gait.shape[0]/6) #6 events in each stride

    HSR_times = gait['Time'][gait.EventType == 'HSR']
    HSL_times = gait['Time'][gait.EventType == 'HSL']
    TOR_times = gait['Time'][gait.EventType == 'TOR']
    TOL_times = gait['Time'][gait.EventType == 'TOL']
    
    #Taking only 30-60 seconds of the trial for cleaner results for mean trajectory     
    stride_start_index_clean = np.where(HSR_times>30)[0][0]
    stride_stop_index_clean = np.where(HSR_times<60)[0][-1]
    
    #For four events of interest, calculate the closest times from RAWDATA.csv file 
    #Closest to TOL times in raw data 
    HSR_times_raw = [raw['Time'][raw['Time']>HSR_times.iloc[i]].iloc[0] for i in range(stride_count)]
    HSL_times_raw = [raw['Time'][raw['Time']>HSL_times.iloc[i]].iloc[0] for i in range(stride_count)]
    TOR_times_raw = [raw['Time'][raw['Time']>TOR_times.iloc[i]].iloc[0] for i in range(stride_count)]
    TOL_times_raw = [raw['Time'][raw['Time']>TOL_times.iloc[i]].iloc[0] for i in range(stride_count)]
    
    #For each stride in current person-current trial 
    #HSR-TOL-MidSSR-HSL-TOR-MidSSL
    #plt.figure(figsize = (6,9))
    line1s_x, line1s_y = [], []
    line2s_x, line2s_y = [], []
    line3s_x, line3s_y = [], []
    line4s_x, line4s_y= [], []
    IPs_x, IPs_y = [], [] #Intersection points (x, y) for each stride 

    #for idx in range(stride_start_index_clean, stride_stop_index_clean): #Stride index for each person-each trial #use for mean
    for idx in range(0, stride_count): #Use for all strides 
        try:
            stride_idx = (raw['Time']>=HSR_times_raw[idx]) & (raw['Time']<HSR_times_raw[idx+1])
            COP1_idx = (raw['Time']>=HSR_times_raw[idx]) & (raw['Time']<TOL_times_raw[idx])
            line1 = raw[['COPX', 'COPY']][COP1_idx] - raw[['COPX', 'COPY']][stride_idx].mean()
            line1s_x.append(list(line1['COPX'].values))
            line1s_y.append(list(line1['COPY'].values))

            COP2_idx = (raw['Time']>=HSL_times_raw[idx]) & (raw['Time']<TOR_times_raw[idx])
            line2 = raw[['COPX', 'COPY']][COP2_idx] - raw[['COPX', 'COPY']][stride_idx].mean()
            line2s_x.append(list(line2['COPX'].values))
            line2s_y.append(list(line2['COPY'].values))

            COP3_idx = (raw['Time']>=TOL_times_raw[idx]) & (raw['Time']<HSL_times_raw[idx])
            line3 = raw[['COPX', 'COPY']][COP3_idx] - raw[['COPX', 'COPY']][stride_idx].mean()
            line3s_x.append(list(line3['COPX'].values))
            line3s_y.append(list(line3['COPY'].values))

            COP4_idx = (raw['Time']>=TOR_times_raw[idx]) & (raw['Time']<HSR_times_raw[idx+1])
            line4 = raw[['COPX', 'COPY']][COP4_idx] - raw[['COPX', 'COPY']][stride_idx].mean()
            line4s_x.append(list(line4['COPX'].values))
            line4s_y.append(list(line4['COPY'].values))

        
            IP_x, IP_y = intersection(line1['COPX'].values, line1['COPY'].values, line2['COPX'].values, line2['COPY'].values)
            IPs_x.append(IP_x)
            IPs_y.append(IP_y)
            
            '''
            #Plotting for each stride 
            plt.plot(line1.iloc[:,0].values, line1.iloc[:, 1].values, 'c-', alpha = 0.4)
            plt.plot(line2.iloc[:,0].values, line2.iloc[:, 1].values, 'c-', alpha = 0.4)
            plt.plot(line3.iloc[:,0].values, line3.iloc[:, 1].values, 'c-', alpha = 0.4)
            plt.plot(line4.iloc[:,0].values, line4.iloc[:, 1].values, 'c-', alpha = 0.4)
            #plt.plot(IP_x, IP_y, 'yo')
            '''
        except: #For the last stride when HSR of next stride is unavailable 
            continue 
    '''
    #For plotting the clean mean trajectory, we use normalization w.r.t time 
    line1_mean = np.array([pd.DataFrame(line1s_x).iloc[:, :301].interpolate(method='linear', limit_direction='forward', axis=1).mean(), 
                           pd.DataFrame(line1s_y).iloc[:, :301].interpolate(method='linear', limit_direction='forward', axis=1).mean()])
    line2_mean = np.array([pd.DataFrame(line2s_x).iloc[:, :271].interpolate(method='linear', limit_direction='forward', axis=1).mean(), 
                           pd.DataFrame(line2s_y).iloc[:, :271].interpolate(method='linear', limit_direction='forward', axis=1).mean()])
    line3_mean = np.array([pd.DataFrame(line3s_x).iloc[:, :231].interpolate(method='linear', limit_direction='forward', axis=1).mean(), 
                           pd.DataFrame(line3s_y).iloc[:, :231].interpolate(method='linear', limit_direction='forward', axis=1).mean()])
    line4_mean = np.array([pd.DataFrame(line4s_x).iloc[:, :211].interpolate(method='linear', limit_direction='forward', axis=1).mean(), 
                       pd.DataFrame(line4s_y).iloc[:, :211].interpolate(method='linear', limit_direction='forward', axis=1).mean()])
    plt.plot(line1_mean[0], line1_mean[1], 'k-', linewidth=4, label = None)
    plt.plot(line2_mean[0], line2_mean[1], 'k-', linewidth=4, label = None)
    plt.plot(line3_mean[0], line3_mean[1], 'k-', linewidth=4, label = None)
    plt.plot(line4_mean[0], line4_mean[1], 'k-', linewidth=4, label = 'average trajectory')    
    plt.plot(np.mean(pd.DataFrame(IPs_x))[0], np.mean(pd.DataFrame(IPs_y))[0], 'gd')
    plt.xlim((-0.25,0.25))
    plt.ylim((-0.5, 0.5))
    #plt.xlabel('Position of center of pressure in x-direction')
    #plt.ylabel('Position of center of pressure in y-direction')
    #plt.title('Butterfly diagram: PwMS ID ' + str(pid+1) + ', Trial Walking & Talking (WT)')
    #plt.savefig(path + '..\\butterfly\\' + 'pwms'+str(pid+1)+'_trial'+str(trial_id)+'.jpg')
    plt.legend()
    plt.show()
    '''
    
    return IPs_x, IPs_y

### Computing the intersection points 

In [118]:
#One with PatientID, TrialID, Mean(x), Mean(y), SD(x), SD(y)
df1 = pd.DataFrame()
#Another with Patient ID, Trial ID, x, y, (x-mean(x))^2, (y-mean(y))^2
df2 = pd.DataFrame()

for index in range(0, 30): #30 controls in trial W
    pid = control_ids[index]
    IPs_x, IPs_y = butterfly_plots(index)

    IPs_x = pd.DataFrame(IPs_x)[0]
    IPs_y = pd.DataFrame(IPs_y)[0]
    #Append NaN at the end to compensate for last stride not computed 
    IPs_x  = IPs_x.append(pd.Series([np.nan]), ignore_index = True)
    IPs_y = IPs_y.append(pd.Series([np.nan]), ignore_index = True)
    IPs_x_abs = IPs_x.abs()
    IPs_y_abs = IPs_y.abs()
    mean_x = IPs_x.mean()
    mean_y = IPs_y.mean()
    mean_x_abs = IPs_x_abs.mean()
    mean_y_abs = IPs_y_abs.mean()
    sd_x = IPs_x.std()
    sd_y = IPs_y.std()
    df1 = df1.append([[pid, mean_x, mean_y, mean_x_abs, mean_y_abs, sd_x, sd_y]], ignore_index=True)
    squared_diff_x = (IPs_x- mean_x)**2
    squared_diff_y = (IPs_y- mean_y)**2
    temp_df = pd.DataFrame([[pid]*len(IPs_x),list(IPs_x.values), list(IPs_y.values), 
                            list(IPs_x_abs.values), list(IPs_y_abs.values), list(squared_diff_x.values), 
                            list(squared_diff_y.values)]).T
    df2 = df2.append(temp_df, ignore_index= True)

df1.columns = ['PID', 'ButterflyMean_x', 'ButterflyMean_y', 'ButterflyMean_x_abs', 'ButterflyMean_y_abs', 
               'ButterflySD_x', 'ButterflySD_y']
df2.columns = ['PID', 'Butterfly_x', 'Butterfly_y', 'Butterfly_x_abs', 'Butterfly_y_abs', 
               'ButterflySQ_x', 'ButterflySQ_y']

Time recorded for PID  0  is  177.737999995797
Time recorded for PID  1  is  151.326000030829
Time recorded for PID  2  is  173.248000025191
Time recorded for PID  3  is  154.892000031556
Time recorded for PID  4  is  168.88599999700898
Time recorded for PID  5  is  151.83399999641
Time recorded for PID  6  is  151.979999996406
Time recorded for PID  7  is  169.98599999598
Time recorded for PID  8  is  155.390000017652
Time recorded for PID  9  is  153.972000031368
Time recorded for PID  10  is  180.95199999572102
Time recorded for PID  11  is  141.199999996661
Time recorded for PID  12  is  168.19199999602301
Time recorded for PID  13  is  152.349999996397
Time recorded for PID  14  is  171.123999995953
Time recorded for PID  15  is  200.94799999524798
Time recorded for PID  16  is  178.44999999578
Time recorded for PID  17  is  171.06999999595502
Time recorded for PID  18  is  216.238000001033
Time recorded for PID  19  is  197.54599999532903
Time recorded for PID  20  is  166.065999

In [119]:
#Saving 2 csvs
#One with PatientID, TrialID, Mean(x), Mean(y), SD(x), SD(y)
df1.to_csv(path+'..\\..\\extracted_features\\ButterflyMeanSD_30controlsTrialW.csv')
#Another with Patient ID, Trial ID, x, y, (x-mean(x))^2, (y-mean(y))^2
df2.to_csv(path+'..\\..\\extracted_features\\ButterflyFeatures_30controlsTrialW.csv')

### Foot progression angles

In [122]:
#Foot progression angle - right foot
def FPA_right(x1, x2, y1, y2):
    return np.arctan2((y2 - y1), (x2 - x1)) / math.pi *  180

#Left foot progression angle 
def FPA_left(x1, x2, y1, y2):
    return np.arctan2((y2 - y1), (x2 - x1)) / math.pi * 180 * (-1)

In [123]:
#Foot Progression Calculator function given Patient ID
def FPA_calculator(pid):
    indices_complete, gait, raw = cleaning(pid)

    stride_count = int(gait.shape[0]/6) #6 events in each stride

    #Exact times of HSR
    HSR_times = gait['Time'][gait.EventType == 'HSR']
    #Exact times of TOR 
    TOR_times = gait['Time'][gait.EventType == 'TOR']
    #Exact times of HSL
    HSL_times = gait['Time'][gait.EventType == 'HSL']
    #Exact times of TOL 
    TOL_times = gait['Time'][gait.EventType == 'TOL']

    #X for HSR
    HSR_X = gait['X'][gait.EventType == 'HSR']
    #X for TOR 
    TOR_X = gait['X'][gait.EventType == 'TOR']
    #X for HSL
    HSL_X = gait['X'][gait.EventType == 'HSL']
    #X for TOL
    TOL_X = gait['X'][gait.EventType == 'TOL']

    #Y for HSR
    HSR_Y = gait['Y'][gait.EventType == 'HSR']
    #Y for TOR 
    TOR_Y = gait['Y'][gait.EventType == 'TOR']
    #Y for HSL
    HSL_Y = gait['Y'][gait.EventType == 'HSL']
    #Y for TOL
    TOL_Y = gait['Y'][gait.EventType == 'TOL']

    #For four events of interest, calculate the closest times from RAWDATA.csv file 
    HSR_times_raw = [raw['Time'][raw['Time']>HSR_times.iloc[i]].iloc[0] for i in range(stride_count)]
    TOR_times_raw = [raw['Time'][raw['Time']>TOR_times.iloc[i]].iloc[0] for i in range(stride_count)]
    HSL_times_raw = [raw['Time'][raw['Time']>HSL_times.iloc[i]].iloc[0] for i in range(stride_count)]
    TOL_times_raw = [raw['Time'][raw['Time']>TOL_times.iloc[i]].iloc[0] for i in range(stride_count)]

    rely_progR = []
    rely_footR = []
    rely_progL = []
    rely_footL = []

    for idx in range(0, stride_count): #Use for all strides for each person, each trial
        try:
            #For Right Foot
            #Relative y indices for HSR(i-1) to HSR(i) 
            rely_prog_idxR = (raw['Time']>=HSR_times_raw[idx]) & (raw['Time']<HSR_times_raw[idx+1]) #Progression vector 

            #Relative y indices for HSR(i) to TOR(i) 
            rely_foot_idxR = (raw['Time']>=HSR_times_raw[idx+1]) & (raw['Time']<TOR_times_raw[idx+1]) #Foot direction vector

            #Relative_y or Belt Speed = Speed*dt = Area under the speed curve *dt for HSR(i-1) to HSR(i)
            rely_progR.append(np.trapz(raw['Speed'][rely_prog_idxR])*0.002) 

            #Relative_y or Belt Speed = Speed*dt = Area under the speed curve *dt for HSR(i) to TOR(i)
            rely_footR.append(np.trapz(raw['Speed'][rely_foot_idxR])*0.002) 

            #For Left Foot 
            #Relative y indices for HSL(i-1) to HSL(i) 
            rely_prog_idxL = (raw['Time']>=HSL_times_raw[idx]) & (raw['Time']<HSL_times_raw[idx+1]) #Progression vector 

            #Relative y indices for HSL(i) to TOL(i+1) 
            rely_foot_idxL = (raw['Time']>=HSL_times_raw[idx+1]) & (raw['Time']<TOL_times_raw[idx+2]) #Foot direction vector

            #Relative_y or Belt Speed = Speed*dt = Area under the speed curve *dt for HSL(i-1) to HSL(i)
            rely_progL.append(np.trapz(raw['Speed'][rely_prog_idxL])*0.002) 

            #Relative_y or Belt Speed = Speed*dt = Area under the speed curve *dt for HSL(i) to TOL(i+1)
            rely_footL.append(np.trapz(raw['Speed'][rely_foot_idxL])*0.002) 
        except:
            pass

    #Right Foot 
    #HSR_Y after adding the relative y correspoding to previous HSR
    rel_HSR_Y = HSR_Y[1:].values+np.array(rely_progR) 
    #TOR_Y after adding the relative y corresponding to same foot HSR starting from foot 2
    rel_TOR_Y = TOR_Y[1:].values+np.array(rely_footR) 

    #Left Foot
    #HSL_Y of foot 2(stride 2) after adding the relative y correspoding to previous HSL (first and last one cannot be used)
    #First one cannot be used since we need first 2 to compute Heel-Heel angle
    #Last one cannot be used since we need TOL of next stride (which doesn't exist) to compute HeelLeft-Toe Left angle 
    rel_HSL_Y = HSL_Y[1:-1].values+np.array(rely_progL) 
    #TOL_Y of foot 2 (stride 3) after adding the relative y corresponding to same foot HSL starting from foot 2 
    #(first 2 will not be used)
    rel_TOL_Y = TOL_Y[2:].values+np.array(rely_footL) 

    #Right FPA
    #HSR-NextHSR angle with horizontal axis 
    HH_R = list(map(FPA_right, HSR_X[:-1].values, HSR_X[1:].values, HSR_Y[:-1].values, rel_HSR_Y))
    #HSR-TOR angle with horizontal axis starting from second HSR 
    HT_R = list(map(FPA_right, HSR_X[1:].values, TOR_X[1:].values, HSR_Y[1:].values, rel_TOR_Y))
    #Right Foot Progression angle = angle of HH with horixontal axis - angle of HT with horizontal axis
    rightFPA = np.array(HH_R)-np.array(HT_R)

    #Left FPA
    #HSL-NextHSL angle with horizontal axis 
    HH_L = list(map(FPA_left, HSL_X[:-2].values, HSL_X[1:-1].values, HSL_Y[:-2].values, rel_HSL_Y))
    #HSL-TOL angle with horizontal axis starting from second HSL
    HT_L = list(map(FPA_left, HSL_X[1:-1].values, TOL_X[2:].values, HSL_Y[1:-1].values, rel_TOL_Y))
    #Right Foot Progression angle = angle of HH with horixontal axis - angle of HT with horizontal axis
    leftFPA = np.array(HH_L)-np.array(HT_L)
    
    #Right Foot
    #Convert in-consequetive gait cycles' angles to NaN 
    stride_idx = np.array(indices_complete[::6][1:]) - np.array(indices_complete[::6][:-1])
    #If this difference is not 6, that means the valid strides is not in consequent order, hence, we cannot compute angles 
    rightFPA[np.where(stride_idx!=6)[0]] = np.nan
    #Total length must match the stride count 
    #For RFPA, append NaN at the beginning
    rightFPA = np.insert(rightFPA, 0, np.nan)

    #Left Foot
    #If this difference is not 6, that means the valid strides is not in consequent order, hence, we cannot compute angles 
    for x in np.where(stride_idx!=6)[0]:
        if ((x-1)>=0): #For 0th index, only 0 should be Nan, else, x-1 and x should be Nan
            leftFPA[x-1] = np.nan
        try: 
            #For last index, LeftFPA has length no. of strides-2, so, for the last stride to be non-consequetive, 
            #there will be no current LeftFPA value to be nullified 
            leftFPA[x] = np.nan
        except:
            pass
    #Total length must match the stride count 
    #For LFPA, append NaN at the beginning and at the end
    leftFPA = np.insert(leftFPA, 0, np.nan)
    leftFPA= np.append(leftFPA, np.nan)
    
    return leftFPA, rightFPA

### Creating the dataframe and storing in .csv

In [124]:
#Dataframe with Patient ID, Trial ID, (Right FPA, Left FPA for each stride)
df = pd.DataFrame()

for index in range(0, 30): #30 controls in Trial W
    pid = control_ids[index]
    leftFPA, rightFPA = FPA_calculator(index)

    temp_df = pd.DataFrame(data = np.array([[pid]*len(leftFPA), leftFPA, rightFPA]).T)
    df = df.append(temp_df, ignore_index= True)
df.columns = ['PID', 'LeftFPA', 'RightFPA']

Time recorded for PID  0  is  177.737999995797
Time recorded for PID  1  is  151.326000030829
Time recorded for PID  2  is  173.248000025191
Time recorded for PID  3  is  154.892000031556
Time recorded for PID  4  is  168.88599999700898
Time recorded for PID  5  is  151.83399999641
Time recorded for PID  6  is  151.979999996406
Time recorded for PID  7  is  169.98599999598
Time recorded for PID  8  is  155.390000017652
Time recorded for PID  9  is  153.972000031368
Time recorded for PID  10  is  180.95199999572102
Time recorded for PID  11  is  141.199999996661
Time recorded for PID  12  is  168.19199999602301
Time recorded for PID  13  is  152.349999996397
Time recorded for PID  14  is  171.123999995953
Time recorded for PID  15  is  200.94799999524798
Time recorded for PID  16  is  178.44999999578
Time recorded for PID  17  is  171.06999999595502
Time recorded for PID  18  is  216.238000001033
Time recorded for PID  19  is  197.54599999532903
Time recorded for PID  20  is  166.065999

In [125]:
df.to_csv((path+'..\\..\\extracted_features\\FPAs_30controlsTrialW.csv'))

### Combining all features

In [127]:
#Combining all features together, including butterfly features and angles to create a final raw dataframe 
#Also inserting the labels 
FPAs = pd.read_csv(path+'..\\..\\extracted_features\\FPAs_30controlsTrialW.csv')
butterfly = pd.read_csv(path+'..\\..\\extracted_features\\ButterflyFeatures_30controlsTrialW.csv')
whole_df = pd.concat([FPAs[['LeftFPA', 'RightFPA']], butterfly[['Butterfly_x_abs', 'Butterfly_y_abs', 
                                                                'ButterflySQ_x', 'ButterflySQ_y']], final_df], axis = 1)  
#Saving to .csv 
whole_df.to_csv(path + '..\\..\\extracted_features\\gait_features.csv')

In [128]:
print (whole_df.shape)

(4011, 31)
